# Q3

In [1]:
import numpy as np
from numpy.linalg import inv,matrix_rank,det
from sklearn.preprocessing import PolynomialFeatures

X = np.array([[1.5, 2.0],[-0.5, 1.8], [2.1, -1.2], [0, 0.5]])
y = np.array([4.3, 1.2, 3.7, 0.8])
order = 2
poly = PolynomialFeatures(order)
X_poly = poly.fit_transform(X)
# check if X^T * X is invertible
def check_inverse_rank(matrix, tol=1e-15):
    rank = matrix_rank(matrix)
    print("matrix rank is : "+ str(rank))
    print("matrix size is : "+ str(matrix.shape))

    if matrix.shape[0] == matrix.shape[1]:
       if rank == matrix.shape[0]:
           print("matrix is invertible")
       else:
           print("matrix is not invertible")
    else:
       print("matrix is not square, hence not invertible")

    return (rank == matrix.shape[0]) and (matrix.shape[0] == matrix.shape[1])
check_inverse_rank(X_poly.T @ X_poly)

matrix rank is : 4
matrix size is : (6, 6)
matrix is not invertible


False

# Q8

In [2]:
X = np.array([[6, 14, 55],[3, 18, 72], [7, 11, 65], [2, 16, 58]])
y = np.array([7,6.5,7.5,4.5])
lamda = 0.1
order = 2
poly = PolynomialFeatures(order)
X_poly = poly.fit_transform(X)
# check if X^T * X is invertible
if check_inverse_rank(X_poly.T @ X_poly):
    w = inv(X_poly.T @ X_poly) @ X_poly.T @ y
else:
    reg_L = lamda*np.identity(X_poly.shape[1])
    w = inv(X_poly.T @ X_poly + reg_L) @ X_poly.T @ y

# prediction
x_test = np.array([[4, 19, 80]])
x_test_poly = poly.fit_transform(x_test)
y_test = x_test_poly @ w
print(y_test)

matrix rank is : 4
matrix size is : (10, 10)
matrix is not invertible
[8.25224645]


# Q9

In [3]:
def multi_logistic_cost_gradient(X, W, Y):

    # Compute prediction, cost and gradient based on cross entropy
    z = X @ W
    exp_z = np.exp(z)
    pred_Y = exp_z / np.sum(exp_z, axis=-1, keepdims=True)
    cost   = np.sum(-(Y * np.log(pred_Y)))/X.shape[0]
    gradient = X.T @ (pred_Y-Y) / X.shape[0]

    return pred_Y, cost, gradient

In [4]:
from sklearn.preprocessing import OneHotEncoder
# Step 0: prepare data
X = np.array([[1.2, -0.4, 0.8],[-0.6, 2.0, -0.5],[0.3, -1.2, 1.7],[2.1, 0.5, -0.8]])
poly = PolynomialFeatures(1)
P = poly.fit_transform(X)

y_raw = np.array([[3], [1], [2], [3]])
onehot_encoder=OneHotEncoder(sparse_output=False)
Y = onehot_encoder.fit_transform(y_raw)

lr = 0.1
# Step 1: initialize weights
W = np.array([[0, 0, 0],[0.02, -0.01, 0.03],[-0.05, 0.04, 0.01],[0.03, 0.02, -0.02]])

# Step 2: perform one step of gradient descent
pred_Y, cost, gradient = multi_logistic_cost_gradient(P, W, Y)
W = W - lr*gradient
print("Updated weights after 1st iteration: \n", W)

Updated weights after 1st iteration: 
 [[-0.00836671 -0.00813688  0.0165036 ]
 [-0.02076001 -0.02594286  0.08670287]
 [-0.00440691  0.00065335  0.00375356]
 [ 0.00516703  0.05342267 -0.02858971]]


# Q10

In [5]:
def ReLU(x):
    return np.maximum(0, x)
def Softmax(x):
    return np.exp(x) / np.sum(np.exp(x), axis=-1, keepdims=True) 

In [6]:
X = np.array([[1.2, -0.4, 0.8],[-0.6, 2.0, -0.5],[0.3, -1.2, 1.7],[2.1, 0.5, -0.8]])
poly = PolynomialFeatures(1)
P = poly.fit_transform(X)

y_raw = np.array([[3], [1], [2], [3]])
onehot_encoder=OneHotEncoder(sparse_output=False)
Y = onehot_encoder.fit_transform(y_raw)

lr = 0.1
# Step 1: initialize weights
W1_0 = W2_0 = np.array([[0, 0, 0],[0.02, -0.01, 0.03],[-0.05, 0.04, 0.01],[0.03, 0.02, -0.02]])

### Forward Pass ###
# first layer output
L1=ReLU(P @ W1_0)
print("first layer output:", L1)

# second layer output
# Column of 1s (2x1)
ones = np.ones((L1.shape[0], 1))
# Concatenate along columns (axis=1)
L1_with_ones = np.hstack((ones, L1))
L2=Softmax(L1_with_ones @ W2_0)
print("second layer output:", L2)

### Backward Pass ###
# Error at Second layer
E2 = L2-Y
print("Error at Second layer:", E2)
# Gradient at Second layer
G2 = (L1_with_ones.T @ E2)/X.shape[0]
print("Gradient at Second layer:", G2)
# Weights update at Second layer
W2_1 = W2_0 - lr * G2
print("Weights update at Second layer:", W2_1)
# Error at First layer
E1 = E2 @ W2_0[1:].T * (L1 > 0) #(L1>0)=(Z1>0), which is the derivative of ReLU.
print("Error at First layer:", E1)
# Gradient at First layer
G1 = (P.T @ E1)/X.shape[0]
print("Gradient at First layer:", G1)
# Weights update at First layer
W1_1 = W1_0 - lr * G1
print("Weights update at First layer:", W1_1)

first layer output: [[0.068 0.    0.016]
 [0.    0.076 0.012]
 [0.117 0.    0.   ]
 [0.    0.    0.084]]
second layer output: [[0.33359104 0.33285795 0.33355101]
 [0.33214751 0.33438706 0.33346543]
 [0.33359278 0.33242392 0.33398331]
 [0.33389325 0.3336129  0.33249384]]
Error at Second layer: [[ 0.33359104  0.33285795 -0.66644899]
 [-0.66785249  0.33438706  0.33346543]
 [ 0.33359278 -0.66757608  0.33398331]
 [ 0.33389325  0.3336129  -0.66750616]]
Gradient at Second layer: [[ 0.08330615  0.08332046 -0.1666266 ]
 [ 0.01542864 -0.01386802 -0.00156062]
 [-0.0126892   0.00635335  0.00633584]
 [ 0.00634257  0.00934046 -0.01568303]]
Weights update at Second layer: [[-0.00833061 -0.00833205  0.01666266]
 [ 0.01845714 -0.0086132   0.03015606]
 [-0.04873108  0.03936466  0.00936642]
 [ 0.02936574  0.01906595 -0.0184317 ]]
Error at First layer: [[-0.01665023 -0.          0.02999387]
 [-0.          0.05010276 -0.02001714]
 [ 0.02336712 -0.         -0.        ]
 [-0.         -0.          0.03003918]